## Image classification using MobileNet
In essence, this code creates a model that uses a pre-trained MobileNet network for feature extraction and then adds fully connected layers for classification. The fully connected layer is the part that will be further trained to adapt to the specific data of your classification problem, while the MobileNet layers remain unchanged and are used to extract features from the images. 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.layers import Flatten, Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator , img_to_array, load_img
from keras.applications.mobilenet import MobileNet, preprocess_input 
from keras.losses import categorical_crossentropy

In [4]:
# Create a MobileNet model with an input shape of 224x224 pixels and 3 color channels (RGB).
# The parameter include_top=False means that the fully connected classification layers are not included.
model = MobileNet(input_shape=(224, 224, 3), include_top=False)

# To prevent further training of the MobileNet layers, all layers in the model are set as non-trainable.
for layer in model.layers:
    layer.trainable = False

# Add a Flatten layer to convert the model's output into a one-dimensional vector.
x = Flatten()(model.output)

# Add a Dense layer with 7 units and a 'softmax' activation function for final classification.
x = Dense(units=7, activation='softmax')(x)

# Create a new model that combines the MobileNet architecture with the added layers for classification.
model = Model(model.input, x)
